In [48]:
#text_file = "eng-ban.txt"
text_file = "ban-eng.txt"
# To open the text file we need to encode the text. Here, we use 'utf8' encoding
with open(text_file, encoding="utf8") as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    bangla, english = line.split(" >>> ")
    bangla = "[start] " + bangla + " [end]"
    text_pairs.append((english, bangla))

In [49]:
import random
for i in range(len(text_pairs)):
    print(random.choice(text_pairs));

('About hundrad million people speaks in Bangla.', '[start] প্রায় এক কোটি মানুষ বাংলায় কথা বলে. [end]')
('Bangla is my mother tounge.', '[start] বাংলা আমার মাতৃভাষা. [end]')
('I do not know him.', '[start] আমি উনাকে চিনি না. [end]')
('About hundrad million people speaks in Bangla.', '[start] প্রায় এক কোটি মানুষ বাংলায় কথা বলে. [end]')
('Bangladesh is a developing country.', '[start] বাংলাদেশ একটি উন্নয়নশীল দেশ. [end]')
('I do not know him.', '[start] আমি উনাকে চিনি না. [end]')
('About hundrad million people speaks in Bangla.', '[start] প্রায় এক কোটি মানুষ বাংলায় কথা বলে. [end]')
('Bangladesh is a south-asian country.', '[start] বাংলাদেশ দক্ষিন এশিয়ার একটি দেশ. [end]')
('It is a Bangla to English translation file.', '[start] এটা বাংলা থেকে ইংরেজি অনুবাদের নথি. [end]')
('It is a Bangla to English translation file.', '[start] এটা বাংলা থেকে ইংরেজি অনুবাদের নথি. [end]')
('Bangladesh is a developing country.', '[start] বাংলাদেশ একটি উন্নয়নশীল দেশ. [end]')
('Do you ever been in Bangladesh?',

In [25]:
import random
random.shuffle(text_pairs)
total_data_length = len(text_pairs)
num_val_samples = int(0.15 * total_data_length)

In [26]:
num_train_samples = (total_data_length - (2 * num_val_samples))
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples]